In [1]:
import pandas as pd
# import packages (some of this is from the liet library)
import sys
sys.path.append('/Users/geba9152/LIET/liet/')
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import rnap_lib_data_proc as dp
from liet_res_class import FitParse
from liet_res_class import fitparse_intersect
import numpy as np
import time
from collections import OrderedDict
import rnap_lib_data_proc as dp
from functools import reduce

Date: 05/16/2024
    
Objective: run mumerge on individual SRRs per celltype

Batches I want to run on: 
```
mcf7-batch
kbm7-batch
lymphoblast-batch
hek239-batch
hela-batch
hep2-batch
k562-batch
```

mumerge input: 
``` 
#file	sampid	group
/path/to/sample1.bed	sample1 control
/path/to/sample2.bed    sample2	control
/path/to/sample3.bed    sample3	treatment
/path/to/sample4.bed    sample4	treatment 
```

**input beds will be centered on mT and have a width of sT for now**

# Step 1: load in data
- this function is in the process of being made into a python function I pull in from another script
- very similar to code in `00_plotting`

In [2]:
# create iterb to iterate over
batch_names = ['hct116-batch','hela-batch','k562-batch','hek239-batch','hep2-batch','lymphoblast-batch','kbm7-batch']

# batch_names = ['beas2b-batch']
iterb = pd.DataFrame(batch_names, columns=['batch'])

cell_types = ['HCT116', 'HeLa', 'K562', 'HEK239T-HEK239','HEp2','lymphoblast','KBM7']

iterb['cell type'] = cell_types

iterb

,batch,cell type
0,hct116-batch,HCT116
1,hela-batch,HeLa
2,k562-batch,K562
3,hek239-batch,HEK239T-HEK239
4,hep2-batch,HEp2
5,lymphoblast-batch,lymphoblast
6,kbm7-batch,KBM7


In [3]:
def pull_out_params_individual_samples(iterb, refpoint):
    
    # set a base path
    base_path = "/scratch/Users/geba9152/LIET-summer2024/"
    
    # Initialize dictionary to store the results
    results_dict = defaultdict(pd.DataFrame)
    
    # loop through iterb table to get batches
    for index, row in iterb.iterrows():
        batch = row['batch']
        print(f"Processing batch: {batch}")
        
        # define paths
        lietpath = f"{base_path}{batch}/results/{refpoint}/*.liet"
        
        lietfiles = glob.glob(lietpath)
            
        for lietfile in lietfiles:
            # call fitparse function to get model params
            fit_parser = FitParse(lietfile)

            data = {
                "Gene": fit_parser.genes,
                "mL": fit_parser.mL,
                "mL_std": fit_parser.mL_std,
                "sL": fit_parser.sL,
                "sL_std": fit_parser.sL_std,
                "tI": fit_parser.tI,
                "tI_std": fit_parser.tI_std,
                "mT": fit_parser.mT,
                "mT_std": fit_parser.mT_std,
                "sT": fit_parser.sT,
                "sT_std": fit_parser.sT_std,
                "w": fit_parser.w,
                "w_std": fit_parser.w_std,
                "mL_a": fit_parser.mL_a,
                "mL_a_std": fit_parser.mL_a_std,
                "sL_a": fit_parser.sL_a,
                "sL_a_std": fit_parser.sL_a_std,
                "tI_a": fit_parser.tI_a,
                "tI_a_std": fit_parser.tI_a_std,
                "w_a": fit_parser.w_a,
                "w_a_std": fit_parser.w_a_std,
            }

            data_df = pd.DataFrame(data)

            ann = pd.read_csv("/scratch/Users/geba9152/LIET-summer2024/meta-celltype-comparison/annotation/lietanns/chr1-6-3p-UTR.liet.ann", sep = "\t", header = None)
            ann = ann[[3,4]]
            ann.columns = ['Gene','Length']

            df = ann.merge(data_df, on = 'Gene')
            df['mT'] = df['mT'] - df['Length']

            SRR = os.path.basename(lietfile).replace('.liet',"")
            
            if batch not in results_dict:
                results_dict[batch] = {}

            if SRR not in results_dict[batch]:
                results_dict[batch][SRR] = pd.DataFrame()

            results_dict[batch][SRR] = pd.concat([results_dict[batch][SRR], df], axis=0, ignore_index=True)
    
    return results_dict
                
refpoint = "tight-sT/"
samp_results_dict = pull_out_params_individual_samples(iterb, refpoint)
samp_results_dict

Processing batch: hct116-batch
Processing batch: hela-batch
Processing batch: k562-batch
Processing batch: hek239-batch
Processing batch: hep2-batch


CHECK LINE: ['SLC9A1|NR_046474.2: fitting error']


Processing batch: lymphoblast-batch
Processing batch: kbm7-batch


CHECK LINE: ['NUCKS1|NM_022731.5: fitting error']


defaultdict(pandas.core.frame.DataFrame,
            {'hct116-batch': {'SRR8867629':                      Gene  Length     mL  mL_std      sL  sL_std       tI  \
              0       RPL22|NM_000983.4   14576  74.73    8.39   55.82    8.59  1095.85   
              1     ENO1|NM_001353346.3   17687  10.00    9.14  213.66    7.86   205.99   
              2    UBE4B|NM_001410744.1  148282  75.80    9.86   50.39   11.67   406.51   
              3         SRM|NM_003132.3    5430  45.29   12.65   23.13   15.61   444.63   
              4       EFHD2|NM_024329.6   20452  50.73    9.16   12.40   12.10   325.39   
              ..                    ...     ...    ...     ...     ...     ...      ...   
              158      NUS1|NM_138459.5   35259  66.41   14.70  132.91   19.73   442.54   
              159     RPS12|NM_001016.4    2996  70.91    4.28   35.11    5.14   623.48   
              160    ABRACL|NM_021243.3   14558   5.82   24.37  316.31   26.88   415.43   
              161  

In [14]:
def make_mumerge_beds(samp_results_dict, center, width, cell_types):
    
    for batch, values in samp_results_dict.items(): 
        print(batch)
        for srr in values:
            print(srr)
            # place srr values into a dataframe
            bed_data = pd.DataFrame(values[srr])
            
            bed_data = bed_data[['Gene','mT','sT']]
            
            ann = pd.read_csv("/scratch/Users/geba9152/LIET-summer2024/meta-celltype-comparison/annotation/lietanns/chr1-6-3p-UTR.liet.ann", sep = "\t", header = None)
            ann.columns = ['chr','start','stop','Gene','len','strand']
            
            bed_data = bed_data.merge(ann, on = 'Gene')
            
            bed_data['mT-genomic-coords'] = bed_data.apply(lambda row: row['start'] + row['mT'] if row['strand'] == "+" else row['stop'] - row['mT'], axis=1)
            
            bed_data['start-bed'] = bed_data['mT-genomic-coords'] - bed_data['sT']
            bed_data['stop-bed'] = bed_data['mT-genomic-coords'] + bed_data['sT']
            bed_data['.'] = '.'
            
            bed_data = bed_data[['chr','start-bed','stop-bed','Gene','.','strand']]
            
            bed_data.to_csv(f"/scratch/Users/geba9152/LIET-summer2024/mumerge-srrs/beds/{batch}-{srr}-mT-center-sT-width.bed", header = None, sep = "\t", index = None)   
                    

make_mumerge_beds(samp_results_dict, 'mT', 'sT',cell_types)

hct116-batch
SRR8867629
SRZ6290499
SRR8867636
SRZ6290523
SRR8867628
SRR8867637
SRZ6290507
SRZ6290515
SRR9304731
SRR9304730
hela-batch
SRR8478997
SRR8478992
SRR8478996
SRR8478993
k562-batch
SRR8669163
SRR4454567
SRR11793825
SRR4454568
SRR5364304
SRZ1554311
SRR8137173
SRR8669162
SRR11793826
SRR12083665
SRR12083664
SRR5364303
hek239-batch
SRR7988490
SRR6927839
SRR6026781
SRR6026782
SRR7988488
hep2-batch
SRR8957130
SRR8957137
SRR8957133
SRR8957138
SRR8957134
SRR8957131
SRR8957132
SRR8957135
lymphoblast-batch
SRR6727941
SRR6727951
SRR6727954
SRR6727944
SRR6727946
SRR6727956
SRR6727959
SRR6727953
SRR6727955
SRR6727945
SRR6727942
SRR6727948
SRR6727950
SRR6727958
SRR6727952
SRR6727957
SRR6727943
SRR6727947
SRR6727949
kbm7-batch
SRZ10354608
SRZ10354602
SRR10354624
SRR10354625
SRZ10354610
SRZ10354600


In [16]:
df = pd.read_csv("/scratch/Users/geba9152/LIET-summer2024/mumerge-srrs/input.txt", sep = "\t", header = None)
df

,0,1,2
0,hct116-batch-SRR8867628-mT-center-sT-width.bed,sample1,celltype1
1,hct116-batch-SRR8867629-mT-center-sT-width.bed,sample2,celltype1
2,hct116-batch-SRR8867636-mT-center-sT-width.bed,sample3,celltype1
3,hct116-batch-SRR8867637-mT-center-sT-width.bed,sample4,celltype1
4,hct116-batch-SRR9304730-mT-center-sT-width.bed,sample5,celltype1
...,...,...,...
59,lymphoblast-batch-SRR6727955-mT-center-sT-widt...,sample15,celltype7
60,lymphoblast-batch-SRR6727956-mT-center-sT-widt...,sample16,celltype7
61,lymphoblast-batch-SRR6727957-mT-center-sT-widt...,sample17,celltype7
62,lymphoblast-batch-SRR6727958-mT-center-sT-widt...,sample18,celltype7


In [13]:
samp_results_dict['hct116-batch']['SRR8867629']

,Gene,Length,mL,mL_std,sL,sL_std,tI,tI_std,mT,mT_std,...,w,w_std,mL_a,mL_a_std,sL_a,sL_a_std,tI_a,tI_a_std,w_a,w_a_std
0,RPL22|NM_000983.4,14576,74.73,8.39,55.82,8.59,1095.85,49.74,943.65,30.85,...,"[0.23, 0.3, 0.33, 0.14]","[0.01, 0.01, 0.01, 0.01]",67.57,16.44,230.21,30.03,570.79,53.23,"[0.88, 0.12]","[0.02, 0.02]"
1,ENO1|NM_001353346.3,17687,10.00,9.14,213.66,7.86,205.99,15.60,2901.81,47.88,...,"[0.1, 0.68, 0.2, 0.02]","[0.0, 0.01, 0.01, 0.0]",46.27,21.14,308.96,22.98,1752.49,63.95,"[0.95, 0.05]","[0.01, 0.01]"
2,UBE4B|NM_001410744.1,148282,75.80,9.86,50.39,11.67,406.51,29.00,2498.62,158.73,...,"[0.08, 0.61, 0.06, 0.25]","[0.0, 0.02, 0.0, 0.02]",35.39,34.23,634.91,57.42,417.99,65.36,"[0.65, 0.35]","[0.03, 0.03]"
3,SRM|NM_003132.3,5430,45.29,12.65,23.13,15.61,444.63,52.72,1450.35,27.50,...,"[0.12, 0.2, 0.43, 0.25]","[0.01, 0.01, 0.01, 0.02]",77.68,12.86,140.05,24.13,1057.96,49.32,"[0.98, 0.02]","[0.01, 0.01]"
4,EFHD2|NM_024329.6,20452,50.73,9.16,12.40,12.10,325.39,31.34,1913.64,79.93,...,"[0.1, 0.36, 0.42, 0.12]","[0.01, 0.02, 0.01, 0.01]",25.51,35.75,285.88,49.02,309.03,61.06,"[0.56, 0.44]","[0.05, 0.05]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,NUS1|NM_138459.5,35259,66.41,14.70,132.91,19.73,442.54,47.81,2902.72,264.22,...,"[0.08, 0.52, 0.35, 0.05]","[0.01, 0.02, 0.02, 0.01]",43.94,21.71,180.28,16.98,1179.21,69.95,"[0.95, 0.05]","[0.01, 0.01]"
159,RPS12|NM_001016.4,2996,70.91,4.28,35.11,5.14,623.48,16.92,1879.67,15.62,...,"[0.24, 0.01, 0.74, 0.01]","[0.01, 0.0, 0.01, 0.0]",77.50,12.01,195.38,44.07,1128.23,48.81,"[0.96, 0.04]","[0.01, 0.01]"
160,ABRACL|NM_021243.3,14558,5.82,24.37,316.31,26.88,415.43,44.70,1332.96,82.45,...,"[0.35, 0.41, 0.22, 0.02]","[0.02, 0.03, 0.02, 0.0]",-6.32,19.73,305.68,18.06,504.04,37.77,"[0.93, 0.07]","[0.01, 0.01]"
161,HECA|NM_016217.3,45723,58.11,17.92,33.75,25.95,798.57,99.98,3810.81,344.55,...,"[0.18, 0.44, 0.15, 0.23]","[0.01, 0.03, 0.02, 0.03]",9.02,56.23,644.13,84.51,388.39,98.85,"[0.87, 0.13]","[0.05, 0.05]"
